<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Баланс-классов" data-toc-modified-id="Баланс-классов-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Баланс классов</a></span><ul class="toc-item"><li><span><a href="#Изменение-весов-классов" data-toc-modified-id="Изменение-весов-классов-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Изменение весов классов</a></span></li><li><span><a href="#Ресемплирование-с-уменьшением-класса-0" data-toc-modified-id="Ресемплирование-с-уменьшением-класса-0-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Ресемплирование с уменьшением класса 0</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#Классификатор-LogisticRegression" data-toc-modified-id="Классификатор-LogisticRegression-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Классификатор LogisticRegression</a></span></li><li><span><a href="#Классификатор-DecisionTreeClassifier" data-toc-modified-id="Классификатор-DecisionTreeClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Классификатор DecisionTreeClassifier</a></span></li><li><span><a href="#Классификатор-CatBoostClassifier" data-toc-modified-id="Классификатор-CatBoostClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Классификатор CatBoostClassifier</a></span></li><li><span><a href="#Классификатор-SGDClassifier" data-toc-modified-id="Классификатор-SGDClassifier-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Классификатор SGDClassifier</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Итоговый-вывод" data-toc-modified-id="Итоговый-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Итоговый вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

Код выполнялся в Яндекс.Облаке в Data Sphere с бесплатным тарифом на 4000р на 2 месяца, т.к. тренажер вылетал в Яндекс.Практикум. Ячейку ниже можно закомментировать, если не нужна.

## Подготовка

In [ ]:
import pandas as pd 
import numpy as np 

import re
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stopwords = set(nltk_stopwords.words('english'))

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
try:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
except:
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/toxic_comments.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [ ]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
display(df['toxic'].value_counts())
class_ratio = df['toxic'].value_counts()[0] / df['toxic'].value_counts()[1]
class_ratio

0    143346
1     16225
Name: toxic, dtype: int64

8.834884437596301

Классы несбалансированы. Отношение 1:8.83. Проведем поиск лучшего способа балансировки и сравним качество.

- Изменение весов в модели обучения
- Ресемплирование с уменьшением класса 0

Ресемплирование с увеличением класса 1 не будем использовать из-за громоздкого набора данных.

Подготовим признаки и целевой признак перед обучением.

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [ ]:
%%time
df['text'] = df['text'].map(lambda x: clean_text(x))

CPU times: user 7.19 s, sys: 128 ms, total: 7.32 s
Wall time: 7.29 s


In [ ]:
df['text']

0         explanation why the edits made under my userna...
1         d aww he matches this background colour i am s...
2         hey man i am really not trying to edit war it ...
3         more i cannot make any real suggestions on imp...
4         you sir are my hero any chance you remember wh...
                                ...                        
159566    and for the second time of asking when your vi...
159567    you should be ashamed of yourself that is a ho...
159568    spitzer umm theres no actual article for prost...
159569    and it looks like it was actually you who put ...
159570    and i really do not think you understand i cam...
Name: text, Length: 159571, dtype: object

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
%%time
lemmatizer = WordNetLemmatizer()

df['text'] = df['text'].map(lambda x:[lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(x)])\
    .map(lambda x:' '.join(str(e) for e in x))

df['text'][0]

CPU times: user 14min 41s, sys: 1min 27s, total: 16min 8s
Wall time: 16min 5s


'explanation why the edits make under my username hardcore metallica fan be revert they be not vandalism just closure on some gas after i vote at new york doll fac and please do not remove the template from the talk page since i be retire now 89 205 38 27'

In [ ]:
df['text'].head() #лемматизация проведена

0    explanation why the edits make under my userna...
1    d aww he match this background colour i be see...
2    hey man i be really not try to edit war it jus...
3    more i can not make any real suggestion on imp...
4    you sir be my hero any chance you remember wha...
Name: text, dtype: object

Разобьем выборку по отношению 60/20/20. Уменьшим количество кроссвалидаций до 3 из-за размера выборки.

In [ ]:
X = df.drop('toxic', axis=1)
y = df['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=23052022)

print(X_train.shape)
print(X_test.shape)

(111699, 1)
(47872, 1)


In [ ]:
cv_counts = 3

In [ ]:
%%time

lr_pipe = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1,2), min_df=2, max_df=0.9, use_idf=1,
                                              smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
                    ('clf', LogisticRegression(random_state=12345))])

params = {'clf__C': [0.1, 1, 10],
          'clf__class_weight': ['balanced', None]}

lr_grid = GridSearchCV(estimator=lr_pipe, param_grid=params, cv=2, scoring='f1', n_jobs=-1, refit=False)
lr_grid.fit(X_train['text'], y_train)
lr_best_paramms = lr_grid.best_params_

print(lr_best_paramms)
print(lr_grid.best_score_)

{'clf__C': 10, 'clf__class_weight': 'balanced'}
0.7740095524469972
CPU times: user 1.44 s, sys: 1.56 s, total: 3 s
Wall time: 1min 17s


## Обучение

Для обучения выберем следующие модели:

- LogisticRegression

### LGBMClassifier

In [ ]:
%%time

lgb_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), min_df=2, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LGBMClassifier(random_state=23052022))])

params = {
  'clf__n_estimators': [100],
  'clf__learning_rate': [0.15, 0.25],
  'clf__max_depth': [8, 10, -1]}



lgb_grid = GridSearchCV(estimator=lgb_pipe, param_grid=params, cv=2, scoring='f1', n_jobs=-1, refit=False)
lgb_grid.fit(X_train['text'], y_train)
lgb_best_params = lgb_grid.best_params_

print(lgb_best_params)
print(lgb_grid.best_score_)

{'clf__learning_rate': 0.25, 'clf__max_depth': -1, 'clf__n_estimators': 100}
0.7577436446515378
CPU times: user 2.2 s, sys: 3.76 s, total: 5.96 s
Wall time: 3min 59s


In [ ]:
vectorize = TfidfVectorizer(ngram_range=(1,2),
               min_df=2, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)

In [ ]:
X_train = vectorize.fit_transform(X_train['text'])
X_test = vectorize.transform(X_test['text'])

In [ ]:
lr_m = LogisticRegression(C=10, class_weight='balanced', random_state=12345)
lr_m.fit(X_train, y_train)

LogisticRegression(C=10, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=12345, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lgb_m = LGBMClassifier(learning_rate=0.25, max_depth=-1, n_estimators=100, random_state = 12345)
lgb_m.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.25, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=12345, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
def scoring(fitted_model):
    test_pred = fitted_model.predict(X_test)
    test_f1 = f1_score(y_test, test_pred)
    
    print('F1 on test: {:.3f}'.format(test_f1))

In [ ]:
scoring(lr_m)

F1 on test: 0.786


In [ ]:
scoring(lgb_m)

F1 on test: 0.764


### Вывод
К тестовому набору данных перейдут LogisticRegression

## Итоговый вывод
В ходе работы над проектом было сделано:

- Подготовленны данные обучения на моделях.
- Выбран способ баланса классов и поделены данные на обучающую, валидационную и тестовою выборку.
- Обучены модели и выбраны лучшие из них на валидационной выборке.
- Показаны параметры качества моделей.

Исходные данные обладают большим количеством признаков. Созданных столбцов больше, чем записей данных. Так как TF-IDF превращают текст в численные значения, лучшими моделями стали LogisticRegression и SGDClassifier. CatBoostClassifier может показать себя очень хорошо при долгом обучении на данных. В ходе тестов данный классификатор мог обучатся до 5 часов.

На тестовой выбоке по метрике F1 лучше всего себя показал LogisticRegression всего на 0.09. Данная модель обладает больними показателями Precision и Accuracy. Это говорит нам, что токсичные комментарии находятся лучше.

SGDClassifier показал себя лучше в ROC AUC и Recall метриках. Модель способна обработать больше записей.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны